In [1]:
from reinforcement_training import *
from notebook_game_helper import *
from keras.models import load_model

Using Theano backend.


In [ ]:
# lets train a DQN model!
# make the model
hiddens = [{"size":100,"activation":"relu"}]
#            {"size":100,"activation":"relu"},
#           {"size":100,"activation":"relu"}]
# make an optimizer
from keras.optimizers import sgd, RMSprop, Adagrad, Adadelta, Adam
# note to self: DON'T CHANGE THIS UNTIL YOU KNOW WE'RE LEARNING SOMETHING
# optimizer = sgd(lr = 0.0001)
# optimizer_str = "SGD"
# optimizer = Adagrad()
# optimizer_str = "Adagrad"
# optimizer = RMSprop()
# optimizer_str = "RMSprop"
# optimizer = Adadelta()
# optimizer_str = "Adadelta"
optimizer = Adam()
optimizer_str = "Adam"
# model = baseline_model(optimizer, hiddens, ipt_mode = 3, opt_mode = 1)
model = load_model("guided_rl_model.h5")

In [ ]:
# model = baseline_model(optimizer, hiddens, ipt_mode = 3, opt_mode = 1)

In [ ]:
# model.load_weights("guided_rl_model.h5")

In [ ]:
training_game_size_x = 40
training_game_size_y = 30

training_game = HybridNaviGame(training_game_size_y,
                                training_game_size_x,
                                model,
                                tolerance = 1.3)
training_game.setup()

In [ ]:
# training_game.add_wall(length=8, start=(25, 5), step=(-2, 1))
training_game.add_block()

In [ ]:
training_game.Navigator.strategy.mode = 3

In [ ]:
training_episodes = 10000
steps = 5
# train the model
output = train_model(game = training_game,
                model = model,
                episodes = training_episodes,
                steps = steps,
                e_start = .9,
                e_stop = .1)

In [ ]:
# plot learning info
title_str = str(training_game_size_y) + "x" + str(training_game_size_x) + " with " 
title_str += str(training_episodes) + " episodes, " + str(steps) + " steps per episode\n "
title_str += "One hidden layer with 100 units, optimized with " + optimizer_str + "\n"
f, axarr = pl.subplots(3, 1, figsize = (8, 10.5), dpi = 600)

for _, k in enumerate([100, 500, 2000]):
    mean_step = k
    mean_rewards = []
    mean_dists = []
    mean_loss = []
    num_means = int(len(output['distances'])/mean_step/steps)
    steps_per_mean = steps*mean_step
    x = np.linspace(0, training_episodes, num_means)
    for i in range(num_means):
        mean_r = 0
        mean_d = 0
        mean_l = 0
        for j in range(steps_per_mean):
            mean_r += output['rewards'][j + i * steps_per_mean]
            mean_d += output['distances'][j + i * steps_per_mean]
            mean_l += output['loss'][j + i * steps_per_mean]
        mean_r = mean_r / steps_per_mean
        mean_d = mean_d / steps_per_mean
        mean_l = mean_l / steps_per_mean
        mean_rewards.append(mean_r)
        mean_dists.append(mean_d)
        mean_loss.append(mean_l)
    label = str(mean_step) + " Episodes"
    axarr[0].plot(x, mean_loss, label = label)
    axarr[1].plot(x, mean_dists, label = label)
    axarr[2].plot(x, mean_rewards, label = label)

axarr[0].grid(True)
axarr[0].set_title(title_str + 'Mean Loss') 
axarr[1].grid(True)
axarr[1].set_title('Mean Distances from Goal')
axarr[2].grid(True)
axarr[2].set_title('Mean Rewards')
f.subplots_adjust(hspace=0.2)

# axarr[1].plot(x, output['replays'])
# axarr[1].set_title('Replay Loss')
# axarr[2].plot(x2, output['reward_totals'])
# axarr[2].set_title('Total Reward')
# axarr[2].plot(x2, output['distances'])
# axarr[2].set_title('Distance from Goal')

file_str = str(training_game_size_y) + "x" + str(training_game_size_x) + "_" 
file_str += str(training_episodes) + "_" + str(steps) + "_" + str(len(hiddens))
file_str += "_" + optimizer_str
pl.legend()
pl.plot()
pl.savefig("larry_plus_more_with_obstacles_1.png")
pl.show()

In [ ]:
for i in range(5):
    training_game.add_block()

In [ ]:
draw_game(training_game)

In [ ]:
animate_game(training_game, n = 10, save = True)

In [ ]:
model.save("larry_1.h5")

In [ ]:
training_game.Navigator.move(2, 2, False)

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = "images/"
filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# join(mypath, f)
import imageio
images = []
for filename in filenames[1:]:
    images.append(imageio.imread(filename))
imageio.mimsave('larry/20000_x_15/larry_gif_14_larry_maze_d.gif', images)

In [ ]:
training_game.Navigator.strategy.plan_movement()

In [ ]:
model.summary()

In [ ]:
122646 * 32/8

In [ ]:
model.regularizers